## ETL Dataset Steam Games 

Exploración del dataset y limpieza

In [13]:
# Importacion de librerias
import json  
import pandas as pd
#pandas.options.display.max_rows = 15
import numpy as np
import re
import ast
import warnings
warnings.filterwarnings('ignore')

Existen filas que contienen arreglos y estan presentes en las columna "Genres" por lo que se tiene que desagrupar esa columna para poder trabajarlo como un DataFrame

Se verificó que la columna "Tags" es similar a la columna "Genres", lo que nos llevaría a prescindir de esta así como de la columnas: "url", "reviews_url", "specs" y "early_access" que no son explicativas para el Proyecto

In [14]:
dataGames = pd.read_json('output_steam_games.json', lines=True)
dataGames.tail(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,None,"Nikita ""Ghost_RUS""",None,NaN,None,NaN
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,None,Sacada,None,NaN,None,NaN
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,None,Laush Dmitriy Sergeevich,None,NaN,None,NaN
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns",None,NaN,None,NaN
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,NaN,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None,None,None,NaN,None,NaN


Esto convertirá nuestro archivo JSON en un marco de datos de pandas.

In [15]:
rows = []
with open('output_steam_games.json') as json_file:
    for line in json_file:
        rows.append(json.loads(line))

In [16]:
data = pd.json_normalize(rows, record_path=['genres'], meta=['publisher', 'app_name','title','release_date','price','id','developer','items_count','metascore'])
data.shape

(71554, 10)

In [17]:
data.rename(columns={0:'genres'}, inplace=True)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71554 entries, 0 to 71553
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        71554 non-null  object
 1   publisher     59115 non-null  object
 2   app_name      71552 non-null  object
 3   title         71552 non-null  object
 4   release_date  71501 non-null  object
 5   price         68355 non-null  object
 6   id            71552 non-null  object
 7   developer     71204 non-null  object
 8   items_count   0 non-null      object
 9   metascore     5470 non-null   object
dtypes: object(10)
memory usage: 5.5+ MB


In [19]:
data.metascore.unique()

array([nan, 96, 84, 80, 76, 70, 'NA', 69, 81, 75, 72, 66, 67, 77, 91, 89,
       83, 61, 88, 65, 94, 57, 86, 87, 92, 79, 82, 58, 74, 85, 90, 68, 71,
       60, 73, 59, 64, 54, 53, 78, 51, 44, 63, 38, 56, 49, 52, 62, 93, 48,
       34, 95, 43, 55, 24, 46, 41, 20, 39, 45, 35, 47, 40, 36, 50, 32, 37,
       33, 42, 27, 29, 30], dtype=object)

In [20]:
data.tail(5)

,genres,publisher,app_name,title,release_date,price,id,developer,items_count,metascore
71549,Indie,Laush Studio,Russian Roads,Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,NaN,NaN
71550,Racing,Laush Studio,Russian Roads,Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,NaN,NaN
71551,Simulation,Laush Studio,Russian Roads,Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich,NaN,NaN
71552,Casual,SIXNAILS,EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,4.99,658870,"xropi,stev3ns",NaN,NaN
71553,Indie,SIXNAILS,EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,4.99,658870,"xropi,stev3ns",NaN,NaN


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71554 entries, 0 to 71553
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        71554 non-null  object
 1   publisher     59115 non-null  object
 2   app_name      71552 non-null  object
 3   title         71552 non-null  object
 4   release_date  71501 non-null  object
 5   price         68355 non-null  object
 6   id            71552 non-null  object
 7   developer     71204 non-null  object
 8   items_count   0 non-null      object
 9   metascore     5470 non-null   object
dtypes: object(10)
memory usage: 5.5+ MB


Capitalizando la primera letra de cada palabra

In [22]:
data['publisher'] = data['publisher'].str.title()
data['genres'] = data['genres'].str.title()
data['title'] = data['title'].str.title()
data['app_name'] = data['app_name'].str.title()
data['developer'] = data['developer'].str.title()

Vericando si hay registros duplicados

In [23]:
data[data.duplicated(keep=False)].size

20

In [24]:
# Remover registros duplicados

data = data.drop_duplicates()
data.reset_index(drop=True,inplace=True)

In [25]:
data[data.duplicated(keep=False)].size

0

In [26]:
data.shape

(71553, 10)

In [27]:
data.isnull().sum()

genres              0
publisher       12439
app_name            2
title               2
release_date       53
price            3199
id                  2
developer         350
items_count     71553
metascore       66084
dtype: int64

In [28]:
data['genres'].fillna('Sin Dato', inplace=True)
data['publisher'].fillna('Sin Dato', inplace=True)
data['app_name'].fillna('Sin Dato', inplace=True)
data['title'].fillna('Sin Dato', inplace=True)
data['release_date'].fillna('', inplace=True)
data['metascore'].fillna('', inplace=True)
data['developer'].fillna('Sin Dato', inplace=True)


In [29]:
data.isnull().sum()

genres              0
publisher           0
app_name            0
title               0
release_date        0
price            3199
id                  2
developer           0
items_count     71553
metascore           0
dtype: int64

Se prescinde de la columna 'release_date' y se toma el Año para la creación de una nueva columna


In [30]:
data['release_date'] = pd.to_datetime(data['release_date'], format='%Y-%m-%d', errors='coerce')
data['release_year'] = data['release_date'].dt.year
data['release_year'] = data['release_year'].astype('Int64')

In [31]:
data.drop(columns=['release_date'], inplace=True)


In [32]:
data['release_year'].fillna(0, inplace=True)

Verificando los valores de la Columna price debería ser Float


In [33]:
# Extracción del string de la columna price
data['free_play'] = data['price'].str.extract(r'([A-Za-z\s]+)')

In [34]:
data['free_play'] = data['free_play'].replace(np.nan, 'Sin Dato')
data['metascore'] = data['metascore'].replace('NA', 0)

In [35]:
#Conversion a numerico
data['price'] = pd.to_numeric(data['price'], errors='coerce')
data['metascore'] = pd.to_numeric(data['metascore'], errors='coerce')

In [36]:
data['price'] = data['price'].replace(np.nan, 0)
data['metascore'] = data['metascore'].replace(np.nan, 0)

In [37]:
data['id'] = data['id'].replace(np.nan, 0)

In [38]:
data.rename(columns={'id': 'item_id'}, inplace=True)

In [39]:
data = data.drop(data[data['release_year']==0].index)

In [40]:
data = data.reset_index(drop=True)

In [41]:
data.isnull().sum()

genres              0
publisher           0
app_name            0
title               0
price               0
item_id             0
developer           0
items_count     70703
metascore           0
release_year        0
free_play           0
dtype: int64

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70703 entries, 0 to 70702
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        70703 non-null  object 
 1   publisher     70703 non-null  object 
 2   app_name      70703 non-null  object 
 3   title         70703 non-null  object 
 4   price         70703 non-null  float64
 5   item_id       70703 non-null  object 
 6   developer     70703 non-null  object 
 7   items_count   0 non-null      object 
 8   metascore     70703 non-null  float64
 9   release_year  70703 non-null  Int64  
 10  free_play     70703 non-null  object 
dtypes: Int64(1), float64(2), object(8)
memory usage: 6.0+ MB


In [43]:
data.tail(10)

,genres,publisher,app_name,title,price,item_id,developer,items_count,metascore,release_year,free_play
70693,Simulation,Ghost_Rus Games,Colony On Mars,Colony On Mars,1.99,773640,"Nikita ""Ghost_Rus""",NaN,0.0,2018,Sin Dato
70694,Strategy,Ghost_Rus Games,Colony On Mars,Colony On Mars,1.99,773640,"Nikita ""Ghost_Rus""",NaN,0.0,2018,Sin Dato
70695,Casual,Sacada,Logistical: South Africa,Logistical: South Africa,4.99,733530,Sacada,NaN,0.0,2018,Sin Dato
70696,Indie,Sacada,Logistical: South Africa,Logistical: South Africa,4.99,733530,Sacada,NaN,0.0,2018,Sin Dato
70697,Strategy,Sacada,Logistical: South Africa,Logistical: South Africa,4.99,733530,Sacada,NaN,0.0,2018,Sin Dato
70698,Indie,Laush Studio,Russian Roads,Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,NaN,0.0,2018,Sin Dato
70699,Racing,Laush Studio,Russian Roads,Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,NaN,0.0,2018,Sin Dato
70700,Simulation,Laush Studio,Russian Roads,Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,NaN,0.0,2018,Sin Dato
70701,Casual,Sixnails,Exit 2 - Directions,Exit 2 - Directions,4.99,658870,"Xropi,Stev3Ns",NaN,0.0,2017,Sin Dato
70702,Indie,Sixnails,Exit 2 - Directions,Exit 2 - Directions,4.99,658870,"Xropi,Stev3Ns",NaN,0.0,2017,Sin Dato


In [44]:
print(f"Numero de filas duplicadas = {data.duplicated().sum()}")

Numero de filas duplicadas = 0


In [45]:
### Create a Pickle file using serialization 
import pickle
pickle_out = open("steamGames.pkl","wb")
pickle.dump(data, pickle_out)
pickle_out.close()